In [1]:
import sys
path1 = '/Users/timkrausz/Documents/GitHub/DA_maze_paper/DA/Modules/'
path2 = '/Users/timkrausz/Documents/GitHub/DA_maze_paper/RL/Modules/'
sys.path += [path1,path2]

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()
plt.style.use('default')
plt.style.use("seaborn-poster")
from multi_rat_da import *
from hexLevelAnalyses import *
from valPropRats import *
%matplotlib qt

In [23]:
valDaDf = pd.read_csv("/Volumes/Tim/Photometry/10MfRatDataSet/"+"valDaDf.csv")

In [4]:
valDaDf.loc[valDaDf.tri.diff()!=0,].shape[0]

13381

### load in hex-level df with port q values saved

In [3]:
vrats = ValIterRats(None)
loadpath = "/Volumes/Tim/Photometry/10MfRatDataSet/"
datName = "hexLevelDf_cornerHexCorrectAlignment"#"hexLevelDf_includeCornerHex"#"hexLevelDf"
vrats.df = reduce_mem_usage(pd.read_csv(loadpath+datName+".csv"))
#vrats.df.drop(["acc"],axis=1,inplace=True)
vrats.directory_prefix = loadpath


Memory usage of dataframe is 54.99 MB
Memory usage after optimization is: 12.60 MB
Decreased by 77.1%


### add in Q values if needed

In [4]:
create_triframe(vrats)
vrats.get_portQvals(qtype="port",level="rat")

### remove rats/fibers with no significant positive ramping

In [5]:
vrats.df = vrats.df.loc[vrats.df.rat!="IM-1292",]
seshInfo = pd.read_csv(vrats.directory_prefix+"sessionTable.csv")
seshInfo.loc[:,"ratLoc"] = seshInfo.rat + ':'+ seshInfo.fiberloc
vrats.seshInfo = seshInfo
del seshInfo
vrats.seshInfo.loc[vrats.seshInfo.rat=="IM-1532","fiberloc"] = "NAcc-Core-Right"

seshsToRemove = vrats.seshInfo.loc[vrats.seshInfo.ratLoc=="IM-1478:NAcc-Core-Right","session"].unique()
vrats.df = vrats.df.loc[~(vrats.df.session.isin(seshsToRemove)),]

### load transition matrices and barrier info

In [6]:
vrats.directory_prefix = vrats.phot_directory_prefix
vrats.load_tmats()
vrats.get_barIDs()
vrats.directory_prefix = loadpath

briefly switching directory_prefix to new Nas...
directory_prefix back to nas 8
briefly switching directory_prefix to new Nas...
directory_prefix back to nas 8
briefly switching directory_prefix to new Nas...
directory_prefix back to nas 8
briefly switching directory_prefix to new Nas...
directory_prefix back to nas 8
briefly switching directory_prefix to new Nas...
directory_prefix back to nas 8
briefly switching directory_prefix to new Nas...
directory_prefix back to nas 8
briefly switching directory_prefix to new Nas...
directory_prefix back to nas 8
briefly switching directory_prefix to new Nas...
directory_prefix back to nas 8
briefly switching directory_prefix to new Nas...
directory_prefix back to nas 8
briefly switching directory_prefix to new Nas...
directory_prefix back to nas 8
briefly switching directory_prefix to new Nas...
directory_prefix back to nas 8
briefly switching directory_prefix to new Nas...
directory_prefix back to nas 8
briefly switching directory_prefix to ne

In [9]:
dropInds = vrats.df.loc[(vrats.df.rat=="IM-1272")&(vrats.df.date==8282020)&(vrats.df.block==1)].index
vrats.df.drop(dropInds,axis=0,inplace=True)

### optimze gamma to fit DA in each session

In [7]:
vrats.get_optValIterGams()

100%|██████████| 70/70 [26:43<00:00, 22.91s/it]   


- save plot with optimal gamma values for each session

In [8]:
fig = plt.figure()
hist = plt.hist(vrats.opt_gams,bins=100)
plt.xlabel("Optimal gamma",fontweight="bold",fontsize="xx-large")
plt.ylabel("Session count",fontweight="bold",fontsize="xx-large")
plt.tight_layout()
fig.savefig(vrats.directory_prefix+"da_optimized_gammas4vIter.pdf")

- create table with gamma for each rat and session

In [9]:
gamsByRat = pd.DataFrame(np.array([vrats.opt_gams,vrats.df.loc[vrats.df.session.diff()!=0,"rat"],\
                                   vrats.df.loc[vrats.df.session.diff()!=0,"session"]]).T,columns=["gamma","rat","session"])
gamsByRat.gamma = gamsByRat.gamma.astype(float)
pd.pivot_table(data=gamsByRat,values="gamma",index=["rat","session"],aggfunc="mean").to_csv(vrats.directory_prefix+"da_optimized_gammas4vIter.csv")
pd.pivot_table(data=gamsByRat,values="gamma",index=["rat"],aggfunc="mean").to_csv(vrats.directory_prefix+"da_optimized_gammas4vIter_byRat.csv")

- plot gamma distributions for each rat

In [10]:
fig = plt.figure(figsize=(6,3.7))
sns.barplot(x='rat',y='gamma',data=gamsByRat,color='lightgrey',ci=None)
sns.stripplot(x='rat',y='gamma',data=gamsByRat,color='k',marker='o',size=6,ls='-',lw=4,edgecolor='darkblue')
plt.ylabel("DA-optimized "+r"$\gamma$")
plt.xlabel("Rat ID")
plt.xticks(rotation=-45)
plt.tight_layout()
fig.savefig(vrats.directory_prefix+"da_optimized_gammasByRat4vIter.pdf")

- create dataframe for plots and regression analysis in R

In [11]:
add_withinBlockTriNumberToDf(vrats)
vrats.create_optValDf()
vrats.valDaDf.loc[vrats.valDaDf.notnull().all(axis=1),].to_csv(vrats.directory_prefix+"valDaDf.csv")
vrats.valDaDf.tail()

,DA,vel,pairedHexState,tri,trial,block,session,Value,rat
243431,-0.037964,63.78125,78,1,195,4,105,0.0,IM-1532
243432,0.330322,67.37500,95,1,195,4,105,0.0,IM-1532
243433,0.051514,74.43750,109,1,195,4,105,0.0,IM-1532
243434,0.273193,83.56250,119,1,195,4,105,0.0,IM-1532
243435,-0.356934,35.87500,123,1,195,4,105,0.0,IM-1532


## visualize regression results

- load and plot regression results

In [14]:
valRegSum = pd.read_csv(vrats.directory_prefix+"valDASum.csv")
valRegCoefs = pd.read_csv(vrats.directory_prefix+"valDACoefsByRat.csv")

In [12]:
from photometryQuantifications import plotRegSigLevel

In [15]:
valRegSum

,Unnamed: 0,Estimate,Std. Error,df,t value,Pr(>|t|)
0,(Intercept),-0.195940,0.022434,8.411733,-8.734253,0.000017
1,Value,0.632991,0.098617,8.225430,6.418700,0.000182
2,vel_scaled,6.410088,1.115850,8.046388,5.744582,0.000423


In [16]:
valRegCoefs.loc[:,["Value","vel_scaled"]]

,Value,vel_scaled
0,0.740186,3.895717
1,0.563786,8.303748
2,0.624696,5.588554
3,0.772296,5.445895
4,0.360797,9.814714
5,0.301106,8.915033
6,0.414592,8.572166
7,1.202430,0.916787
8,0.717032,6.238182


In [20]:
fig = plt.figure(figsize=(3.7,4))
xvals = ["Value","Velocity"]
#sns.boxplot(data=regCoefs.loc[:,["prev_rwd_taken","prev_rwd_alt"]],color='lightgrey')#,alpha=.5)
plt.bar([0,1],valRegSum.loc[1:2,"Estimate"].values,alpha=0.5,color='grey')
#sns.lineplot(data=valRegCoefs.loc[:,["Value","vel_scaled"]].T,color='k',alpha=.5,ls='-',lw=1,legend=False,marker='D')
sns.stripplot(data=valRegCoefs.loc[:,["Value","vel_scaled"]],color='k',size=10,marker='D',alpha=1)
#plt.axhline(y=0,ls='--',color='k')
plt.xticks([0,1],xvals)
plotRegSigLevel(valRegSum,0,.85,1)
plotRegSigLevel(valRegSum,1,.85,2)
sigPaired = get_sigRatsPaired_from2samples(valRegCoefs.loc[:,"Value"].values,valRegCoefs.loc[:,"vel_scaled"].values,"greater")
#plot_sigMarkers(sigPaired,0.5,1.0)
#plt.ylim(-10,2.2)
plt.ylabel("DA Regression "+r"$\beta $",fontsize='xx-large')
plt.tight_layout()
fig.savefig(vrats.directory_prefix+"coefBarPlotValVsVel.pdf")

p-value =  0.998046875


In [29]:
fig = plt.figure(figsize=(3.7,6))
xvals = ["Value"]
#sns.boxplot(data=regCoefs.loc[:,["prev_rwd_taken","prev_rwd_alt"]],color='lightgrey')#,alpha=.5)
plt.bar([0,1],valRegSum.loc[1,"Estimate"],alpha=0.5,color='grey')
#sns.lineplot(data=valRegCoefs.loc[:,["Value","vel_scaled"]].T,color='k',alpha=.5,ls='-',lw=1,legend=False,marker='D')
sns.stripplot(data=valRegCoefs.loc[:,["Value"]],color='k',size=10,marker='D',alpha=1)
#plt.axhline(y=0,ls='--',color='k')
plt.xticks([0],xvals)
plotRegSigLevel(valRegSum,0,.85,1)
#plot_sigMarkers(sigPaired,0.5,1.0)
plt.ylim(-1.5,1.5)
plt.ylabel("DA Regression "+r"$\beta $",fontsize='xx-large')
plt.axhline(y=0,ls='--',color='k',lw=2)
plt.tight_layout()
fig.savefig(vrats.directory_prefix+"coefBarPlotVal.pdf")

- load and plot partial r2

In [18]:
partialR2 = pd.read_csv(vrats.directory_prefix+"valDaPartialR2s.csv")

In [30]:
fig = plt.figure(figsize=(4,6))
#plt.bar([0,1],[4.67089,0.82169],color='darkgreen')
plt.bar([0,1],partialR2.values[0,1:]*100,color='k')
plt.xticks([0,1],["Value","Velocity"])
plt.ylabel("Coefficient of partial determination (%)")
plt.tight_layout()
fig.savefig(vrats.directory_prefix+"partialR2ValVsVel.pdf")

### inidividual trial visualizations

In [10]:
vrats.get_distanceToPort(getDistsFromDeadEnds=False)

In [11]:
norm = mpl.colors.Normalize(vmin=-1, vmax=2.5)
#norm = mpl.colors.Normalize(vmin=-1, vmax=4)
DAmap = cm.ScalarMappable(norm=norm,cmap="viridis")

In [71]:
fig = plt.figure(figsize=(8,3.3))

In [43]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

In [57]:
vrats.df.columns

Index(['Unnamed: 0', 'hexlabel', 'port', 'rwd', 'session', 'DA', 'block',
       'trial', 'rat', 'date', 'session_type', 'nom_rwd_chosen', 'vel', 'Q_a',
       'Q_b', 'Q_c', 'lenAB', 'lenBC', 'lenAC', 'nom_rwd_a', 'nom_rwd_b',
       'nom_rwd_c', 'dtop', 'pairedHexState', 'tri'],
      dtype='object')

In [62]:
phexdf.loc[phexdf.statecodes==124]

,0,1,state,statecodes,leftcodes,rightcodes,from_state,to_state
49.38,NaN,38.32,49.38,124,NaN,96.0,49,38


In [86]:
vrats.df.loc[(vrats.df.session==62)&(vrats.df.trial==46),["pairedHexState","hexlabel","port","vel","tri"]]

,pairedHexState,hexlabel,port,vel,tri
192600,-1,49,-100,8.289062,46
192601,123,2,-100,17.312500,46
192602,1,49,-100,14.578125,46
192603,124,38,-100,26.906250,46
192604,96,32,-100,21.859375,46
192605,79,27,-100,33.375000,46
192606,65,22,-100,15.429688,46
192607,52,26,-100,34.656250,46
192608,62,21,-100,20.578125,46
192609,49,25,-100,26.484375,46


In [70]:
def plot_pairedStateValsVsDA(s,b,t,onlyPointing2port=False):
    portz = ['A','B','C']
    plt.clf()
    trial = vrats.valDaDf.loc[(vrats.valDaDf.session==s)&(vrats.valDaDf.block==b)\
                              &(vrats.valDaDf.tri==t),"trial"].values[0]
    nextp = vrats.df.loc[(vrats.df.session==s)&(vrats.df.block==b)&\
                    (vrats.df.tri==t)&(vrats.df.port!=-100),"port"].values[0]
    fromp = vrats.df.loc[(vrats.df.session==s)&(vrats.df.block==b)&\
                                (vrats.df.tri==t-1),"port"].values[0]
    toPort = portz[nextp]
    if onlyPointing2port:
        if len(vrats.sesh_arrows2goal['to'+toPort][s])==126:
            arrows2include = np.where(vrats.sesh_arrows2goal['to'+toPort][s])[0]
        else:
            arrows2include = np.where(vrats.sesh_arrows2goal['to'+toPort][s][b-1])[0]
    else:
        arrows2include = np.arange(126)
    meanHexDa = vrats.valDaDf.loc[(vrats.valDaDf.session==s)&\
                 (vrats.valDaDf.block==b)&(vrats.valDaDf.tri==t)&\
                  (vrats.valDaDf.pairedHexState.isin(arrows2include)),:].groupby(\
                    "pairedHexState").DA.mean()
    
    statevals = vrats.valDaDf.loc[(vrats.valDaDf.session==s)&\
                 (vrats.valDaDf.block==b)&(vrats.valDaDf.tri==t)&\
                  (vrats.valDaDf.pairedHexState.isin(arrows2include)),:].groupby(\
                    "pairedHexState").Value.mean()
    
    statelist = meanHexDa.index.values
    stateDa = meanHexDa.values
    plt.subplot(121)
    plt.title("value map")
    plot_hex_outline(vrats,sesh=s,block=b,ax=plt.gca(),size='sm')
    plot_arrowMapFromStates(statelist,statevals.values)
    plt.axis("off")
    
    plt.subplot(122)
    plt.title("DA")
    plot_hex_outline(vrats,sesh=s,block=b,ax=plt.gca(),size='sm')
    plot_arrowMapFromStates(statelist,stateDa,plotDA=True)
    print("v(chosen) = ",vrats.df.loc[(vrats.df.session==s)&\
                                (vrats.df.tri==t)&(vrats.df.block==b)&\
                                (vrats.df.port!=-100),"nom_rwd_chosen"].values[0])
    plt.axis("off")

In [72]:
interact(plot_pairedStateValsVsDA,s=widgets.IntSlider(
    value=0,
    min=0,
    max=vrats.df.session.max(),
    step=1,
    description='session:'),b = widgets.IntSlider(
    value=1,
    min=1,
    max=8,
    step=1,
    description='block:'), t = widgets.IntSlider(
    value=2,
    min=1,
    max=70,
    step=1,
    description='trial:'))

interactive(children=(IntSlider(value=0, description='session:', max=105), IntSlider(value=1, description='blo…

<function __main__.plot_pairedStateValsVsDA(s, b, t, onlyPointing2port=False)>

In [51]:
fig.savefig(vrats.directory_prefix+"s"+str(9)+"_b"+str(2)+"_t"+str(25)+".pdf")

In [42]:
s = 96#95#73#0
b = 1#1#3
t = 24#27#21#42

plot_pairedStateValsVsDA(s,b,t)
plt.subplots_adjust(hspace=0.000)

v(chosen) =  10.0


- run the following if you want to save this plot

In [49]:
fig.savefig(vrats.directory_prefix+"s"+str(s)+"_b"+str(b)+"_t"+str(t)+".pdf")

- plot example value map with specified gamma and session

In [20]:
statevals

array([0.000967, 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.000967, 0.      , 0.      , 0.      , 0.001075, 0.      ,
       0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.      , 0.001327, 0.      , 0.      , 0.      , 0.001194,
       0.      , 0.001327, 0.001327, 0.      , 0.001194, 0.001473,
       0.001194, 0.      , 0.001075], dtype=float16)

In [47]:
norm = mpl.colors.Normalize(vmin=-1, vmax=2)
DAmap = cm.ScalarMappable(norm=norm,cmap="viridis")


In [48]:
s = 95
t = 42
gamma = 0.72
tmin = 20
vrats.tmat = vrats.sesh_tmats[s]
vrats.bars = vrats.sesh_barIDs[s]
data4vIter = vrats.df.loc[(vrats.df.session==95)&
            (vrats.df.trial.isin(np.arange(tmin,45)))&(vrats.df.port!=-100),]
V = vrats.calcValIter(data4vIter,gamma)
portz = ["A","B","C"]
nextp = data4vIter.loc[data4vIter.trial==t,"port"].values[0]
toPort = portz[nextp]
arrows2include = np.where(vrats.sesh_arrows2goal['to'+toPort][s])[0]
statelist = np.setdiff1d(phexdf.statecodes.unique(),phexdf.loc[(np.isin(phexdf.from_state,vrats.bars+1))|
        (np.isin(phexdf.to_state,vrats.bars+1)),'statecodes'].values)
statelist = statelist[np.isin(statelist,arrows2include)]
statevals = V[t-tmin][statelist]
fig = plt.figure(figsize=(4,3.5))
plt.title("value map")
plot_hex_outline(vrats,sesh=s,block=1,ax=plt.gca(),size='sm')
plot_arrowMapFromStates(statelist,statevals)
plt.axis("off")

(-1.0, 15.0, -1.0, 22.0)

In [46]:
fig.savefig(vrats.directory_prefix+"example_ValueMap_s95b1t"+str(t)+"lessDiscount.pdf")